Chapter 2.b.iv.  Generalization
=======

As we saw, **instantiation** may be used to eliminate universal quantifiers.  Introducing universal quantifiers in a judgment is accomplished via **generalization**.  To a certain extent, these are inverse procedures.  You can instantiate $A \vdash \forall_{x~|~Q_1(x), Q_2(x)} P(x)$ to get $A \cup B \vdash P(t)$ provided $A \cup B \vdash Q_1(t)$ and  $A \cup B \vdash Q_2(t)$ are provable.  You can generalize  $A \cup \{Q_1(x), Q_2(x)\} \vdash P(x)$ to get $A \vdash \forall_{x~|~Q_1(x), Q_2(x)} P(x)$ provided the expressions in $A$ do not have $x$ as a free variable.  Essentially, these convert between implicit and explicit forms of universal quantification.  Free variables of a judgment may be instantiated to anything as long as it is consistent across its the judgment (both sides of the turnstile); in that sense, there is an implicit universal quantification.  With **generalization**, you can make universal quantification of any number of parameters explicit, binding the parameters within the $\forall$ operation.  The only caveat is that assumptions of the original judgment than contain any of the newly quantified variables must move to the right side of the turnstile and become conditions of the $\forall$ operation so they can be properly bound, consistent with the other occurrences.  In our example, $Q_1(x)$ and $Q_2(x)$ were assumptions that became conditions of the new quantifer.

The rules and mechanisms of **generalization** are very simple and versatile.  You can parameterize any variables, over any range of indices, and introduce any conditions (more conditions will only make the statement weaker and therefore no less derivable) with the only caveat that assumptions containing any of the new parameter variables must be in the conditions of the new quantifier.  Assumptions are eliminated when they appear as conditions.  For convenience, any number of nested $\forall$ operations may be introduced in one derivation step, but the effect is the same as if they were introduced one at a time.

In [ ]:
import proveit
from proveit._common_ import f, i, j, n, x
%begin generalization

## Generalization demonstrations

Let us try out the various possibilities.  Let's start from the `general_len` theorem once again.  We will **instantiate** it in a generic fashion and then generalize it in various ways.

In [ ]:
from proveit.core_expr_types.tuples._theorems_ import general_len
general_len

In [ ]:
general_len_inst = general_len.instantiate({j:j}, assumptions=general_len.allConditions())

We will now attempt to generalize $n$.  However, this attempt will fail because we don't have all of the conditions needed to eliminate the assumptions involving $n$ as free.

In [ ]:
from proveit import GeneralizationFailure
try:
    general_len_inst.generalize(n)
    assert False, "Expected an GeneralizationFailure error."
except GeneralizationFailure as e:
    print("Expected error:", e)

Let's just use `general_len_inst.assumptions` as the conditions to make sure we get all of the conditions.

In [ ]:
general_len_forall_n = general_len_inst.generalize(n, conditions=general_len_inst.assumptions)

This is not quite the statement that we had before because $i$, $j$, and $f$ are still free.  That's okay.  They are implicitly universally quantified.  Note that $n \in \mathbb{N^+}$ shows up as a domain of $n$ before `|`.  That is just formatting as a matter of *style*.  Internally, $n \in \mathbb{N^+}$ is just one of the conditions.

In [ ]:
general_len_forall_n.conditions

We can accomplish the same thing by setting a `domain` argument when we call `generalize` and leaving it off of the `conditions` list.  

In [ ]:
from proveit.number import NaturalsPos
general_len_forall_n_v2 = general_len_inst.generalize(n, domain=NaturalsPos, 
                                                      conditions=general_len_inst.assumptions[1:])

In [ ]:
general_len_forall_n_v2 == general_len_forall_n

Let us look at the proof graph.

In [ ]:
general_len_forall_n.proof()

Unlike **instantiation** there is no extra information for the **generalization** derivation step.  It isn't necessary because correctness of a **generalization** is obvious by inspection of the original (step 1) versus new (step 0) judgments.

Now let's generalize `general_len_inst` with all of the original parameters of `general_len`.

In [ ]:
all_params = general_len.allInstanceParams()

In [ ]:
from proveit import ParameterCollisionError
try:
    general_len_inst.generalize(all_params, conditions=general_len_inst.assumptions[1:])
    assert False, "Expected an ParameterCollisionError error."
except ParameterCollisionError as e:
    print("Expected error:", e)

This isn't allowed however because we cannot construct a universal quantifer in which one of the parameters occurs free in parameter indices.  We must nest the quantifiers for $n$ and the other variables whose ranges depend upon $n$.  In this case, instead of given `generalize` a single variable or a list of variables (or variable ranges) as the first argument, we can give it a list of lists of variables (or variable ranges).

In [ ]:
general_len_recovered = general_len_inst.generalize([[n], all_params[1:]], conditions=general_len_inst.assumptions)

However, Prove-It recognizes that this has a short proof by simply invoking the `general_len` theorem.

In [ ]:
general_len_recovered.proof()

To make it more interesting, we shall add in an extraneous condition that only makes the statement weaker.  While we are at it, let's swap out one of the variable ranges to something nonsensical just to show we can.  There are no restrictions as long as they form valid **Lambda** expression parameters (recalling that $\forall$ is an operation that internally operates on a **Lambda** expression).

In [ ]:
from proveit import IndexedVar, varRange
from proveit._common_ import k, l
from proveit.logic import Equals
from proveit.number import one
f_k_to_l = varRange(f, k, l)
general_len_weaker = general_len_inst.generalize([[n], [f_k_to_l], all_params[2:]], 
                                                 conditions=(general_len_inst.assumptions+
                                                            (Equals(IndexedVar(i, one), one),)))

We have weakened the statement by requiring that $i_1 = 1$.  Now let's look at the proof.

In [ ]:
general_len_weaker.proof()

Again, we can check this by simply inspecting the original (step 1) and new (step 0) judgments even though we have introduced multiple, nested $\forall$ operations.

## Automatic generalization

Rather than calling `generalize` manually, **generalization** is typically applied via automation.  Starting from what you want to prove, if there is an "original judgment" that can be generalized to get to this result, **Prove-It** will do so automatically.  Let's demonstrate this with a theorem called `singularConstructiveDilemma`.

In [ ]:
from proveit.logic.boolean.disjunction._theorems_ import singularConstructiveDilemma
from proveit._common_ import A, B, C
singularConstructiveDilemma

In [ ]:
singularConstructiveDilemma.instantiate({C:C}, assumptions = singularConstructiveDilemma.allConditions())

In [ ]:
from proveit.logic import Forall
partially_quantified = Forall((A, B), C, conditions = singularConstructiveDilemma.allConditions())

In [ ]:
partially_quantified.prove().proof()

In [ ]:
%end generalization

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>